In [1]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.0 MB/s 
     |████████████████████████████████| 365 kB 8.2 MB/s 
     |████████████████████████████████| 212 kB 90.0 MB/s 
     |████████████████████████████████| 120 kB 85.9 MB/s 
     |████████████████████████████████| 115 kB 79.3 MB/s 
     |████████████████████████████████| 127 kB 77.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
#@title Colab-specific setup
#@markdown This will ask for you to log into Google Drive.
#@markdown Click on the link and copy over your access token.

import torch
colab = 'google.colab' in str(get_ipython())

if colab:
    !nvidia-smi
    gpu_type = torch.cuda.get_device_name(0)
    if gpu_type != 'Tesla T4':
        raise ValueError("Highly advised to use a T4.")

Mon Sep 19 16:36:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import subprocess, getpass

def get_gpu_usage():
    """
    Returns a dict which contains information about memory usage for each GPU.
    In the following output, the GPU with id "0" uses 5774 MB of 16280 MB.
    253 MB are used by other users, which means that we are using 5774 - 253 MB.
    {
        "0": {
            "used": 5774,
            "used_by_others": 253,
            "total": 16280
        },
        "1": {
            "used": 5648,
            "used_by_others": 253,
            "total": 16280
        }
    }
    """

    # Name of current user, e.g. "root"
    current_user = getpass.getuser()

    # Find mapping from process ids to user names
    command = ["ps", "axo", "pid,user"]
    output = subprocess.check_output(command).decode("utf-8")
    pid_user = dict(row.strip().split()
        for row in output.strip().split("\n")[1:])

    # Find all GPUs and their total memory
    command = ["nvidia-smi", "--query-gpu=index,memory.total", "--format=csv"]
    output = subprocess.check_output(command).decode("utf-8")
    total_memory = dict(row.replace(",", " ").split()[:2]
        for row in output.strip().split("\n")[1:])

    # Store GPU usage information for each GPU
    gpu_usage = {gpu_id: {"used": 0, "used_by_others": 0, "total": int(total)}
        for gpu_id, total in total_memory.items()}

    # Use nvidia-smi to get GPU memory usage of each process
    command = ["nvidia-smi", "pmon", "-s", "m", "-c", "1"]
    output = subprocess.check_output(command).decode("utf-8")
    for row in output.strip().split("\n"):
        if row.startswith("#"): continue

        gpu_id, pid, type, mb, command = row.split()

        # Special case to skip weird output when no process is running on GPU
        if pid == "-": continue

        gpu_usage[gpu_id]["used"] += int(mb)

        # If the GPU user is different from us
        if pid_user[pid] != current_user:
            gpu_usage[gpu_id]["used_by_others"] += int(mb)

    return gpu_usage

In [3]:
import os

os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [6]:
os.environ["HF_ALLOW_CODE_EVAL"]

'1'

In [ ]:
program = "from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:"

In [ ]:
from evaluate import load
code_eval = load("code_eval")
test_cases = ["assert add(2,3)==5"]
candidates = [["def add(a,b): return a*b", "def add(a, b): return a+b"]]
pass_at_k, results = code_eval.compute(references=test_cases, predictions=candidates, k=[1, 2])
print(pass_at_k)

In [ ]:
test_cases = ["assert add(2,3)==5"]
candidates = [["def add(a, b): return a+b", "def add(a, b): return a-b"]]
pass_at_k, results = code_eval.compute(references=test_cases, predictions=candidates, k=[1,2])
print(pass_at_k)

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-nl")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-2B-nl")

In [ ]:
import torch

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
text = "def hello_world():"
input_ids = tokenizer(text, return_tensors="pt").input_ids
model.to(device)
input_ids.to(device)

In [ ]:
generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

In [ ]:
cuda.current_context().get_memory_info()

In [ ]:
cuda.current_context().reset()

In [ ]:
from numba import cuda

for gpu in cuda.gpus:
  with gpu:
      cuda.current_context().deallocations.clear()

In [ ]:
torch.cuda.empty_cache()